In [62]:
import numpy as pd
import pandas as pd
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn import decomposition, ensemble
import textblob, string

In [63]:
df = pd.read_csv("train.csv")

In [64]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [65]:
df.shape

(20800, 5)

In [66]:
## Get the Independent Features
X= df.drop("label",axis=1)
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [67]:
## Get the Dependent features
y=df["label"]
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [68]:
# drop null values
df = df.dropna()
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [69]:
#creating a variable called length for the length of the text
length = []
[length.append(len(str(text))) for text in df['text']]
df['length'] = length
df.head()

#dropping length less than 50
df['text'][df['length'] < 50]

82                                                        
169                                                       
173                                        Guest   Guest  
295                                                       
470                                                       
485            Support Us Calgary Airport Arrivals YYC \n 
592                                                       
871                                                       
901                                                       
936                                                       
950                                                       
1002      Apple HomeKit home security security smart house
1591                                                      
2163                                                      
2228                                                      
2298      Posted: Nov 10th, 2016 by Guest   Guest   Ticker
2382                                                    

In [70]:
df.shape

(18285, 6)

In [71]:
messages=df.copy()
messages.reset_index(inplace=True)
messages.head(10)
messages['title'][6]

'Benoît Hamon Wins French Socialist Party’s Presidential Nomination - The New York Times'

In [72]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub("[^a-zA-Z]", " ", messages["title"][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
    review = " ".join(review)
    corpus.append(review)

In [73]:
corpus[6]

'beno hamon win french socialist parti presidenti nomin new york time'

In [74]:
#bag of words with ngram for more meaning from the document
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
X_count = cv.fit_transform(corpus).toarray()
X_count.shape
y=messages["label"]

In [75]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(X_count, y, test_size=0.33, random_state=0)
cv.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [76]:
## gives you the various parameters of count vectorizers
cv.get_params()
count_df = pd.DataFrame(X_train_count, columns=cv.get_feature_names())
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [77]:
#using TFIDF which down weights unimportant words that appear with high frequency
tfidf_v=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_tfid=tfidf_v.fit_transform(corpus).toarray()
y=messages["label"]

In [78]:
#splitting
X_train_tfid, X_test_tfid, y_train_tfid, y_test_tfid = train_test_split(X_tfid, y, test_size=0.25, random_state=0)
tfidf_v.get_feature_names()[:20]

['abandon',
 'abc',
 'abc news',
 'abduct',
 'abe',
 'abedin',
 'abl',
 'abort',
 'abroad',
 'absolut',
 'abstain',
 'absurd',
 'abus',
 'abus new',
 'abus new york',
 'academi',
 'accept',
 'access',
 'access pipelin',
 'access pipelin protest']

In [79]:
#visualise
count_df = pd.DataFrame(X_train_tfid, columns=cv.get_feature_names())
count_df.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.31335,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Instructions for the modelling: we have 2 sets of training and validation data, 1 is count and the other is tfid, use the respective X and Y train/valid data sets.

I have chosen a 75/25 train test split according the size of the data and googling online. Change it if you find something more suitable.

Preprocessing summary: removed null values, removed blank/short meaningless text. Removed stop words and non-english characters. We end up with 2 set of vectors that we can work with.